<a href="https://colab.research.google.com/github/salarbalou/Data_Analysis_Projects/blob/main/Chemical_Properties_Dictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dotmap # we need dotmap for fast navigation through the dictionary using only dots

In [2]:
import numpy as np #numpy is for calculations
import re # regualr expressions to extract important data from text
import requests # to request and grab the text file from the url
from dotmap import DotMap #here we import dotmap
import string # we need strig to assign a letter to designated value

In [3]:
# Write your compound here;
compound = 'Ethanol'

In [4]:
# this is our dictionary abbrevname definition
extract_single_props = {'Molecular Weight' : 'Mw',
                        'Critical Temperature' : 'Tc',
                        'Critical Pressure' : 'Pc',
                        'Critical Volume' : 'Vc',
                        'Acentric factor' : 'w',
                        'Normal boiling point' : 'Tb',
                        'IG heat of formation' : 'HfIG',
                        'IG Gibbs of formation' : 'GfIG',
                        'Heat of vaporization' : 'HvapNB'}
extract_coeff_props={'Vapor Pressure' : 'Pvap',
                     'Ideal Gas Heat Capacity':'CpIG',
                     'Liquid Heat Capacity' : 'CpL',
                     'Solid Heat Capacity' : 'CpS',
                     'Heat of Vaporization' : 'Hvap',
                     'Liquid Density' : 'rhoL'}

In [5]:
# we are extracting the text from the url containing props and coeffs
baseurl = 'https://raw.githubusercontent.com/profteachkids/chetools/main/data/'
char = 'Props.txt'
text = requests.get(baseurl+compound+char).text

In [6]:
# Making the dict for single_props
single_props_pat = re.compile(r'^ ([A-Z a-z/.()\d]+?)\s*:\s*([0-9.e+-]+)\s+([a-zA-Z0-9-/.()*]*)\s*$', re.MULTILINE) # we are defining the path of the single properties
single_props_dict = dict([single_props[0:2] for single_props in single_props_pat.findall(text)]) # we create a dict for the single propertis here and we remove the units
props = {} # we define an empty dictionary so we can assign it to the any values we want
props = DotMap(props) # props as an empty dicitionary will follow the dotmap protocol for navigation
for k, v in extract_single_props.items(): # we are assigning the fullname and abbrevanem in our abbrevname list to the props dicitonary
    props[v] = float(single_props_dict[k])

In [7]:
# Making the dictionary for single_coeffs
coeffs = {} # again we define an empty dictionary
coeffs = DotMap(coeffs) # which follos dotmap for navigation through it
single_coeffs_pat = re.compile(r'([\w ]+)\s[^\n]*Equation.*?Coeffs:\s*([-0-9. e+-]+)', re.DOTALL) # this is the path for the coeffs we are looking for
single_coeffs_dict = dict(single_coeffs_pat.findall(text)) # we extract and make a dict out of the data of coeffs
for fullname, abbrevname in extract_coeff_props.items(): # the abbrevaname and fullname in coeffs abbrename list must be equalized
    abbrevname = extract_coeff_props[fullname]
    coeffs_values = re.split(r'\s+', single_coeffs_dict[fullname]) # we have to seperate the coeffs_values
    for letter, values in zip(string.ascii_uppercase, coeffs_values): # we assign a letter to each value up to 7 values
         coeffs[abbrevname+letter] = float(values) # we use the previosly described empty dict to call the coeffs.abbrevname+letter and then equilize it to the float of each value

In [8]:
def vapor_pressure(T): # we define a fucntion that returns the value of vapor_pressure or etc...
    return np.exp(coeffs.PvapA + coeffs.PvapB/T + coeffs.PvapC*np.log(T) + coeffs.PvapD*T**coeffs.PvapE)

In [10]:
coeffs.PvapA

74.475